# Proyecto BBDD

## Importamos librerías

In [45]:
import os
import sqlite3
from datetime import datetime
import pandas as pd

import psycopg2

print("Librerías importadas correctamente")

Librerías importadas correctamente


## Importamos y limpiamos los CSV

In [46]:
# Importamos los csv con los datos del ejercicio
df_alumnos_1= pd.read_csv('Data/Clase_1.csv', sep= ";")
df_alumnos_2= pd.read_csv('Data/Clase_2.csv', sep= ";")
df_alumnos_3= pd.read_csv('Data/Clase_3.csv', sep= ";")
df_alumnos_4= pd.read_csv('Data/Clase_4.csv', sep= ";")
df_profesores= pd.read_csv('Data/Claustro.csv', sep= ";")

In [47]:
df_alumnos= pd.concat([df_alumnos_1, df_alumnos_2, df_alumnos_3, df_alumnos_4], ignore_index= True)
df_alumnos.head(5)

Nombre                       Email   Promoción Fecha_comienzo  \
0      Jafet Casals      Jafet_Casals@gmail.com  Septiembre     18/09/2023   
1  Jorge Manzanares  Jorge_Manzanares@gmail.com  Septiembre     18/09/2023   
2     Onofre Adadia     Onofre_Adadia@gmail.com  Septiembre     18/09/2023   
3      Merche Prada      Merche_Prada@gmail.com  Septiembre     18/09/2023   
4      Pilar Abella      Pilar_Abella@gmail.com  Septiembre     18/09/2023   

   Campus Proyecto_HLF Proyecto_EDA Proyecto_BBDD Proyecto_ML  \
0  Madrid         Apto      No Apto          Apto        Apto   
1  Madrid         Apto      No Apto          Apto        Apto   
2  Madrid         Apto         Apto          Apto     No Apto   
3  Madrid         Apto      No Apto       No Apto        Apto   
4  Madrid         Apto      No Apto          Apto        Apto   

  Proyecto_Deployment Proyecto_WebDev Proyecto_FrontEnd Proyecto_Backend  \
0                Apto             NaN               NaN              NaN   
1                Apto             NaN               NaN              NaN   
2                Apto             NaN               NaN              NaN   
3             No Apto             NaN               NaN              NaN   
4                Apto             NaN               NaN              NaN   

  Proyecto_React Proyecto_FullSatck  
0            NaN                NaN  
1            NaN                NaN  
2            NaN                NaN  
3            NaN                NaN  
4            NaN                NaN

In [48]:
# Cambio el nombre de las verticales para que no me de error más tarde

df_profesores['Vertical'] = df_profesores['Vertical'].replace({
    'DS': 'Data Science',
    'FS': 'Full Stack'
})

## Conectamos el Notebook a la base de datos

In [49]:
# Creamos la conexión al servidor de Render

connection = psycopg2.connect(
        host= 'dpg-d41069hr0fns739r6gq0-a.oregon-postgres.render.com',
        database= 'bootcamp_z1q7',
        user= 'bootcamp_z1q7_user',
        password= '4MDOTwmhpav1upPCxzDtPrNStbgNmb2X',
        port= 5432
        ) 

cursor = connection.cursor()

In [50]:
# Con pgAdmin 4 vinculado a Render modificamos las tablas
# Comprobación de que funciona:

cursor.execute("""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public';
""")
tablas = cursor.fetchall()
print("Tablas en la base de datos:")
for tabla in tablas:
    print(tabla[0])

Tablas en la base de datos:
Alumnos
Campus
Vertical
Promo
Clases
Profesores
Matriculas
Asignaciones
Expediente
Expedientes


## Rellenamos tablas

#### Tabla: Alumnos

In [51]:
# Rellenamos cada alumno con nombre e email con un bucle

for index, row in df_alumnos.iterrows():
    cursor.execute('''
    INSERT INTO "Alumnos" ("Nombre", "email")
    VALUES (%s, %s)
    ''', (
        row['Nombre'], 
        row['Email']
    ))

connection.commit()

#### Tabla: Profesores

In [52]:
# Rellenamos cada alumno con nombre e email con un bucle

for index, row in df_profesores.iterrows():
    cursor.execute('''
    INSERT INTO "Profesores" ("Nombre")
    VALUES (%s)
    ''', (
        row['Nombre'],
    ))

connection.commit()

#### Tabla: Asignaciones

In [53]:
# Este pedazo del código ha servido para ver que combiancion de clases de profes frente a alumnos no existía,
# lo que daba un eror en la celda anterior

missing = 0
for i, row in df_profesores.iterrows():
    cursor.execute('''SELECT COUNT(*) FROM "Clases" c
                      JOIN "Vertical" v ON c."ID_Vertical" = v."ID_Vertical"
                      JOIN "Campus" ca ON c."ID_Campus" = ca."ID_Campus"
                      JOIN "Promo" p ON c."ID_Promo" = p."ID_Promo"
                      WHERE v."Vertical"=%s AND ca."Ciudad"=%s AND p."Promocion"=%s''',
                      (row['Vertical'], row['Campus'], row['Promocion']))
    if cursor.fetchone()[0] == 0:
        missing += 1

print(f"Clases no encontradas: {missing}")

Clases no encontradas: 0


## Rollback

Si da error, tirar de aquí, cuando pueda lo implemento en un try except

In [54]:
# SI ALGO FALLA EJECUTAR ESTO ANTES DE SEGUIR
connection.rollback()